In [ ]:
%run -i 'cosim_import.py'

In [ ]:
import time

In [ ]:
caseH = 18

# --- ANDES case ---
dir_path = os.path.abspath('..')
case_path = '/case/ieee39_eva.xlsx'
case = dir_path + case_path
ssa = andes.load(case,
                 setup=False,
                 no_output=True,
                 default_config=False)

# Set output mode as 'manual', turn off TDS progress bar
ssa.TDS.config.save_mode = 'manual'
ssa.TDS.config.no_tqdm = 1

# Set load as constant load.
ssa.PQ.config.p2p = 1
ssa.PQ.config.q2q = 1
ssa.PQ.config.p2z = 0
ssa.PQ.config.q2z = 0
ssa.PQ.pq2z = 0

# Turn on ``numba`` can accelerate TDS.
ssa.config.numba = 1

ss2 = andes.load(case,
                 setup=False,
                 no_output=True,
                 default_config=False)

# Set output mode as 'manual', turn off TDS progress bar
ss2.TDS.config.save_mode = 'manual'
ss2.TDS.config.no_tqdm = 1

# Set load as constant load.
ss2.PQ.config.p2p = 1
ss2.PQ.config.q2q = 1
ss2.PQ.config.p2z = 0
ss2.PQ.config.q2z = 0
ss2.PQ.pq2z = 0

# Turn on ``numba`` can accelerate TDS.
ss2.config.numba = 1

In [ ]:
# --- EV Aggregator ---
sse = ev_ssm(ts=caseH, N=1000, step=1, tp=40,
             lr=0.1, lp=60, seed=2022, name="EVA")
sse.load_A("Aest.csv")

In [ ]:
for i in range(sse.ev.shape[0]):
    ev_dict = {'bus':38, 'gen':'PV_10',
            'qmx': 10, 'qmn':-10, 'pmx':60, 'pmn':-70, 'pcap':1,
            'gammap': 1, 'gammaq': 1, 'En': 10, 'EtaC': 10, 'EtaD': 10, 
            'ddn':0, 'tip':0.02, 'tiq':0.02, 'ialim': 100, 'recflag':0,
            'pqflag':1, 'vrflag':0}
    ssa.add("EV2", ev_dict)
# TODO: alter gammap, gammpaq, pmx, pmn, EtaC, EtaD, En later on
ev_df = sse.ev.copy()
ev_df['p'] = -1 * ev_df['c'] * ev_df['Pc']
ev_df['gammap'] = ev_df['p'] / ev_df['p'].sum()
ev_df['gammaq'] = 1 / ev_df.u.sum()
ev_df['EtaC'] = ev_df['nc']
ev_df['EtaD'] = ev_df['nd']
ev_df['En'] = ev_df['Q'] / 1e3
ev_df['SOCinit'] = ev_df['soc'].replace(1, 0.999)
# ev_df['pmx'] = ev_df['Pc'] / 1e3 / ssa.config.mva
# ev_df['pmn'] = -1 * ev_df['Pc'] / 1e3 / ssa.config.mva
ev_df['u'] = ev_df['u']

ev_idx = ssa.EV2.idx.v
ssa.DG.set(attr='v', idx=ev_idx, src='gammap', value=ev_df['gammap'].values)
ssa.DG.set(attr='v', idx=ev_idx, src='gammaq', value=ev_df['gammaq'].values)
ssa.DG.set(attr='v', idx=ev_idx, src='EtaC', value=ev_df['EtaC'].values)
ssa.DG.set(attr='v', idx=ev_idx, src='EtaD', value=ev_df['EtaD'].values)
ssa.DG.set(attr='v', idx=ev_idx, src='En', value=ev_df['En'].values)
ssa.DG.set(attr='v', idx=ev_idx, src='SOCinit', value=ev_df['SOCinit'].values)
ssa.DG.set(attr='v', idx=ev_idx, src='u', value=ev_df['u'].values)

# TODO: alter PV_10.p0
p0 = ev_df['p'].sum() / 1e3 / ssa.config.mva
ssa.StaticGen.set(src='p0', attr='v', idx='PV_10', value=p0)
ssa.setup()
ssa.PFlow.run()
ssa.TDS.init()

ev0_dict = {'bus':38, 'gen':'PV_10',
        'qmx': 10, 'qmn':-10, 'pmx':60, 'pmn':-70, 'pcap':1,
        'gammap': 1, 'gammaq': 1, 'En': 10, 'EtaC': ev_df['EtaC'].mean(), 'EtaD': ev_df['EtaD'].mean(), 
        'ddn':0, 'tip':0.02, 'tiq':0.02, 'ialim': 100, 'recflag':0,
        'pqflag':1, 'vrflag':0}

ss2.add("EV2", ev0_dict)
ss2.setup()
ss2.StaticGen.set(src='p0', attr='v', idx='PV_10', value=p0)
ss2.PFlow.run()
xx = ss2.TDS.init()

In [ ]:
ta = 0
tb = 0

for end_time in range(1, 1000):
    sse.run(tf=caseH+end_time/3600, Pi=0,
            is_updateA=False, is_rstate=True,
            is_test=False, disable=True)
    ssa.DG.set(attr='v', idx=ev_idx, src='u', value=sse.ev['u'].values)
    ssa.DG.set(attr='v', idx=ev_idx, src='pref0',
               value=list(-1 * sse.ev['c'] * sse.ev['Pc'] / ssa.config.mva / 1e3))
    ss2.DG.set(attr='v', idx='EV2_1', src='pref0',
               value=np.sum(-1 * sse.ev['c'] * sse.ev['Pc'] / ssa.config.mva / 1e3))

    ssa.TDS.config.tf = end_time
    ss2.TDS.config.tf = end_time
    t0 = time.time()
    ssa.TDS.run()
    ta += time.time() - t0
    t0 = time.time()
    ss2.TDS.run()
    tb += time.time() - t0
    
    if ssa.exit_code != 0 | ss2.exit_code != 0:
        break

In [ ]:
ta

In [ ]:
tb

In [ ]:
# calcualte output power
ia = ssa.dae.ts.x[:, ssa.EV2.Ipout_y.a]
va = ssa.dae.ts.y[:, ssa.EV2.v.a]
pa = np.sum(ia * va * ssa.config.mva, axis=1)

i2 = ss2.dae.ts.x[:, ss2.EV2.Ipout_y.a]
v2 = ss2.dae.ts.y[:, ss2.EV2.v.a]
p2 = np.sum(i2 * v2 * ssa.config.mva, axis=1)


In [ ]:
plt.style.use("ieee")
fig, ax = plt.subplots(1, 2, figsize=(10, 4), dpi=200)
plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0.2, hspace=0)
ax[0].plot(ssa.dae.ts.t, pa, label="Case1", color="tab:blue")
ax[0].plot(ssa.dae.ts.t, p2, label="Case2", color="tab:green", linestyle="--")
ax[0].plot(3600*(np.array(sse.tss)-caseH), sse.Ptl, color="tab:orange", linestyle="-.", label="Input")
ax[0].legend()
ax[0].set_xlim([0, end_time])
ax[0].set_xlabel("Time [s]")
ax[0].set_ylabel("Output Power [MW]")
ax[0].legend(frameon=False)
ssa.TDS.plt.plot(ssa.COI.omega, ytimes=60, ax=ax[1], fig=fig, yheader=['Case1'],
                 color="tab:blue", show=False)
ss2.TDS.plt.plot(ss2.COI.omega, ytimes=60, ax=ax[1], fig=fig, yheader=["Case2"],
                 color="tab:green",  linestyles=['--'], show=False)
ax[1].set_ylabel('Freq. [Hz]')
# ax[1].plot(ssa.dae.ts.t, ssa.COI.omega, label="ANDES + Individual", color="tab:blue")
# ax[1].plot(ssa.dae.ts.t, p2, label="ANDES + Aggregated", color="tab:green", linestyle="--")
# ax[1].legend()